In [ ]:
# !unzip /content/drive/MyDrive/gender_dataset_face-20220319T023558Z-001.zip -d /content/drive/MyDrive/gender_dataset_face

In [ ]:
import os
os.makedirs("Model_weights", exist_ok=True)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten

from keras.layers import Dropout
from keras.layers import Dense
from tensorflow.keras import datasets, layers, models
from keras import backend as K
from keras import models
from keras.applications.vgg16 import VGG16


In [ ]:
class SmallerVGGNet:
    @staticmethod
    def vgg_net(width, height, depth, classes):

            conv_base = VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(height, width, depth))
            model = models.Sequential()

            model.add(conv_base)
            model.add(Flatten())

            model.add(Dense(1024, activation='relu'))
            model.add(BatchNormalization())
            model.add(Dropout(0.5))
            model.add(Dense(classes, activation='sigmoid'))
            return model

In [ ]:
import matplotlib
matplotlib.use("Agg")  # Only needed in headless environments like servers
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import img_to_array, to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

# If you have a custom model, make sure the import is correct
# from model.smallervggnet import SmallerVGGNet  # Uncomment if you have this file and model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()

        # First CONV => RELU => POOL layer
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(height, width, depth)))
        model.add(MaxPooling2D(pool_size=(3, 3)))

        # Second CONV => RELU => POOL layer
        model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Third CONV => RELU => POOL layer
        model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        # Flatten and add Fully Connected layers
        model.add(Flatten())
        model.add(Dense(256, activation="relu"))
        model.add(Dropout(0.5))  # Prevent overfitting
        model.add(Dense(classes, activation="softmax"))  # Output layer

        return model


In [ ]:
import glob
import os
import random
epochs = 50
lr = 1e-3
batch_size = 32
img_dims = (96,96,3)

data = []
labels = []

# load image files from the dataset
image_files = [f for f in glob.glob('/content/drive/MyDrive/gender_dataset_face/gender_dataset_face' + "/**/*", recursive=True)
if not os.path.isdir(f)]
random.seed(42)
random.shuffle(image_files)

# create groud-truth label from the image path
for img in image_files:

    image = cv2.imread(img)
    # print(img)
    image = cv2.resize(image, (96,96))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2]
    if label == "woman":
        label = 1
    else:
        label = 0

    labels.append([label])

# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# split dataset for training and validation
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2,
                                                  random_state=2)
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# augmenting datset
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

# build model
# model = SmallerVGGNet.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
#                             classes=2)
model = SmallerVGGNet.vgg_net(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

# compile the model
opt = Adam(learning_rate=lr, decay=lr / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
#checkpoint save
filepath="/content/epochs:{epoch:03d}-val_accuracy:{val_accuracy:.3f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# model.load_model("Model_weights/epochs:009-val_accuracy:0.797.hdf5")

# train the model
history = model.fit(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1,callbacks=callbacks_list)
# model.load_weights("Model_weights/epochs:{epoch:03d}-val_accuracy:{val_accuracy:.3f}.hdf5")

# save the model to disk

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.savefig('/content/drive/My Drive/Interface_model/result.plot')
plt.show()


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(testX)
pred = np.argmax(pred,axis = 1)
y_true = np.argmax(testY,axis = 1)


In [ ]:
CM = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
# plt.show()
fig

In [ ]:
!pip install cvlib

In [ ]:
# from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import img_to_array

from keras.models import load_model
# from keras.utils import get_file
import numpy as np
import argparse
import cv2
import os
!pip install cvlib
import cvlib as cv

In [ ]:
img= '/content/drive/MyDrive/men.jpg'
image = cv2.imread(img)
# cv2.imread('/conte')

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Data Science batch.jpg')

if image is None:
    print("Could not read input image")
    exit()

# load pre-trained model
model = load_model("/content/drive/MyDrive/Colab Notebooks/epochs_044-val_accuracy_0.966.keras")

# detect faces in the image
face, confidence = cv.detect_face(image)

classes = ['man','woman']
# loop through detected faces
for idx, f in enumerate(face):

     # get corner points of face rectangle
    (startX, startY) = f[0], f[1]
    (endX, endY) = f[2], f[3]

    # draw rectangle over face
    cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

    # crop the detected face region
    face_crop = np.copy(image[startY:endY,startX:endX])

    # preprocessing for gender detection model
    face_crop = cv2.resize(face_crop,(96,96))
    face_crop = face_crop.astype("float") / 255.0
    face_crop = img_to_array(face_crop)
    face_crop = np.expand_dims(face_crop, axis=0)

    # apply gender detection on face
    conf = model.predict(face_crop)[0]
    print(conf)
    print(classes)

    # get label with max accuracy
    idx = np.argmax(conf)
    label = classes[idx]

    label = "{}: {:.2f}%".format(label, conf[idx] * 100)

    Y = startY - 10 if startY - 10 > 10 else startY + 10

    # write label and confidence above face rectangle
    if conf[idx] * 100 > 50.0:
        cv2.putText(image, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 1)
# cv2.imwrite('/content/',image)
# display output
# cv2.imshow("gender detection", image)

# # press any key to close window
# cv2.waitKey(0)

# # save output
cv2.imwrite("gender_detection.jpg", image)

# # release resources
# cv2.destroyAllWindows()